In [1]:
%load_ext autoreload
%autoreload 2
%aimport algorithm_utils

In [38]:
from algorithm_utils import *
import general_utils as gu

In [59]:
testing_length = 100
start_index = 5000
df: pd.DataFrame = gu.load_local_data().iloc[start_index:start_index+testing_length].reset_index(drop=True)

In [60]:
zigzag_df = zigzag(df)

In [62]:
pt = gu.PlottingTool()
pt.draw_candlesticks(df)
pt.draw_zigzag(zigzag_df)
pt.draw_zigzag(generate_h_o_zigzag(zigzag_df), title='Higher order Zigzag', color='black')
pt.show()